In [1]:
import os 
import pandas as pd 
import matplotlib.pyplot as plt
from pathlib import Path
import re
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode,  iplot

pd.options.plotting.backend = "plotly"

In [86]:
def performance_data(perf_path):
  output = dict()
  f = open(perf_path.joinpath("performance.log"),"r")
  lines = f.readlines()
  escape_force = False
  nmax = len(lines)
  n = 0
  Big_category = re.compile("Total elapsed time for (.*): *([0-9.,]*) s")
  mpi_all = re.compile("All-to-many operations:")
  cpu = re.compile("Calculation time summary:")
  while n < nmax and not escape_force:
    line = lines[n]
    cat = re.match(Big_category,line)
    if cat:
      Name = cat.groups()[0]
      output[Name] = dict()
      output[Name]["Time_tot"] = float(cat.groups()[1])
      if re.search("gradient",Name):
        n += 2
        line = lines[n]
        grad = re.search("Summary of gradient computations for \"(.*)\"",line)
        while grad and n < nmax:
          name_grad = grad.groups()[0]
          output[Name][name_grad] = dict()
          n += 2
          line = lines[n]
          reconstruct = re.search("Reconstruction type: *(.*)",line)
          if reconstruct:
            output[Name][name_grad]["Reconstruct"] = reconstruct.groups()[0]
          n += 1
          line = lines[n]
          ncalls = re.search("Number of calls: *(\d*)",line)
          if ncalls:
            output[Name][name_grad]["Ncalls"] = int(ncalls.groups()[0])
          n += 1
          line = lines[n]
          time_grad = re.search("Total elapsed time: *([0-9,.]*)",line)
          if time_grad:
            output[Name][name_grad]["Time_grad"] = float(time_grad.groups()[0])
          n += 2
          line = lines[n]
          grad = re.search("Summary of gradient computations for \"(.*)\"",line)
      if re.search("system solvers",Name):
        n += 5
        line = lines[n]
        solv = re.search("Summary of resolutions for \"(.*)\"",line)
        while solv and n<nmax:
          name_solv = solv.groups()[0]
          output[Name][name_solv] = dict()
          n += 2
          line = lines[n]
          var = re.search("Solver type: *(.*)",line)
          if var:
            output[Name][name_solv]["Solver"] = var.groups()[0]
            n += 1
          line = lines[n]
          var = re.search("Preconditioning: *(.*)",line)
          if var:
            output[Name][name_solv]["Preconditioning"] = var.groups()[0]
            n += 1
          line = lines[n]
          var = re.search("Number of setups: *(\d*)",line)
          if var:
            output[Name][name_solv]["Nb_Setup"] = int(var.groups()[0])
            n += 1
          line = lines[n]
          var = re.search("Number of calls: *(\d*)",line)
          if var:
            output[Name][name_solv]["Nb_calls"] = int(var.groups()[0])
            n += 1
          line = lines[n]
          var = re.search("Minimum number of iterations: *(\d*)",line)
          if var:
            output[Name][name_solv]["MinNbIter"] = int(var.groups()[0])
            n += 1
          line = lines[n]
          var = re.search("Maximum number of iterations: *(\d*)",line)
          if var:
            output[Name][name_solv]["MaxNbIter"] = int(var.groups()[0])
            n += 1
          line = lines[n]
          var = re.search("Mean number of iterations: *(\d*)",line)
          if var:
            output[Name][name_solv]["MeanNbIter"] = int(var.groups()[0])
            n += 1
          line = lines[n]
          var = re.search("Total setup time: *([0-9.,]*)",line)
          if var:
            output[Name][name_solv]["TimeSetup"] = float(var.groups()[0])
            n += 1
          line = lines[n]
          var = re.search("Total solution time: *([0-9.,]*)",line)
          if var:
            output[Name][name_solv]["TimeSol"] = float(var.groups()[0])
            n += 1
          line = lines[n]
          solv = re.search("Summary of resolutions for \"(.*)\"",line)
          escape = re.search("------",line)
          while not solv and not escape:
            n += 1
            line = lines[n]
            solv = re.search("Summary of resolutions for \"(.*)\"",line)
            escape = re.search("------",line)
    
    mpi = re.match(mpi_all,line)
    if mpi:
      n +=3
      line = lines[n]
      mpi_data = re.search("Total: *([0-9,.]*) s *([0-9,.]*)  *([0-9,.]*) s *([0-9,.]*)",line)
      output["mpi"] = dict()
      output["mpi"]["mean"] = float(mpi_data.groups()[0])
      output["mpi"]["min"] = float(mpi_data.groups()[1])
      output["mpi"]["max"] = float(mpi_data.groups()[2])
      output["mpi"]["calls"] = int(mpi_data.groups()[3])
    cpu_perf = re.match(cpu,line)
    if cpu_perf:
      Name = "CPU"
      output[Name]=dict()
      n += 2
      line = lines[n]
      var = re.search("User CPU time: *([0-9,.]*) s",line)
      if var:
        output[Name]["User_cpu_time"] = float(var.groups()[0])
      n += 1
      line = lines[n]
      var = re.search("System CPU time: *([0-9,.]*) s",line)
      if var:
        output[Name]["System_cpu_time"] = float(var.groups()[0])
      n += 1
      line = lines[n]
      var = re.search("Total CPU time: *([0-9,.]*) s",line)
      if var:
        output[Name]["Total_cpu_time"] = float(var.groups()[0])
      n += 2
      line = lines[n]
      var = re.search("Elapsed time: *([0-9,.]*) s",line)
      if var:
        output[Name]["Elapsed_time"] = float(var.groups()[0])
      n += 1
      line = lines[n]
      var = re.search("CPU \/ elapsed time *([0-9,.]*)",line)
      if var:
        output[Name]["Ratio_time"] = float(var.groups()[0])
    n+=1
  return output

In [87]:
dico = performance_data(Path("/media/juleslecoustre/Jules/TJunction/STRUCT_SKE/STRUCT_SKE_24ms"))

In [89]:
pd.DataFrame.from_dict(dico['linear equation system solvers'])

,Time_tot,Velocity,Pressure,k,epsilon,TempC,Wall distance
Solver,2300.627,Symmetric Gauss-Seidel,Flexible Conjugate Gradient,Symmetric Gauss-Seidel,Symmetric Gauss-Seidel,Symmetric Gauss-Seidel,Flexible Conjugate Gradient
Nb_Setup,2300.627,3333,3333,3333,3333,3333,1
Nb_calls,2300.627,33143,15033,3913,13298,32260,2
MinNbIter,2300.627,1,1,1,1,3,9
MaxNbIter,2300.627,100,14,100,100,100,12
MeanNbIter,2300.627,27,5,2,2,55,10
TimeSetup,2300.627,0.35,104.773,0.548,0.547,0.528,0.032
TimeSol,2300.627,612.841,476.429,2.237,8.96,420.103,0.331
Preconditioning,2300.627,NaN,Multigrid,NaN,NaN,NaN,Multigrid


In [90]:
pd.DataFrame.from_dict(dico['all gradient computations'])

,Time_tot,[cell mass flux divergence update],[face mass flux update],[momentum],[scalar convection-diffusion],epsilon,k,pressure,pressure_increment,temperature,velocity,wall_distance
Reconstruct,202.409,Least-squares,Least-squares,Least-squares,"Green-Gauss, least-squares gradient face values",Least-squares,Least-squares,Least-squares,Least-squares,"Green-Gauss, least-squares gradient face values",Least-squares,"Green-Gauss, least-squares gradient face values"
Ncalls,202.409,33399,6666,3333,2,19964,10579,6666,3333,38928,56484,3338
Time_grad,202.409,27.647,5.352,3.702,0.003,17.9,9.644,8.661,3.134,61.052,60.078,5.238


In [91]:
pd.DataFrame.from_dict(dico)

,all gradient computations,linear equation system solvers,controller,mpi,CPU
Time_tot,202.409,2300.627,0.117,NaN,NaN
[cell mass flux divergence update],"{'Reconstruct': 'Least-squares', 'Ncalls': 333...",NaN,NaN,NaN,NaN
[face mass flux update],"{'Reconstruct': 'Least-squares', 'Ncalls': 666...",NaN,NaN,NaN,NaN
[momentum],"{'Reconstruct': 'Least-squares', 'Ncalls': 333...",NaN,NaN,NaN,NaN
[scalar convection-diffusion],"{'Reconstruct': 'Green-Gauss, least-squares gr...",NaN,NaN,NaN,NaN
epsilon,"{'Reconstruct': 'Least-squares', 'Ncalls': 199...","{'Solver': 'Symmetric Gauss-Seidel', 'Nb_Setup...",NaN,NaN,NaN
k,"{'Reconstruct': 'Least-squares', 'Ncalls': 105...","{'Solver': 'Symmetric Gauss-Seidel', 'Nb_Setup...",NaN,NaN,NaN
pressure,"{'Reconstruct': 'Least-squares', 'Ncalls': 666...",NaN,NaN,NaN,NaN
pressure_increment,"{'Reconstruct': 'Least-squares', 'Ncalls': 333...",NaN,NaN,NaN,NaN
temperature,"{'Reconstruct': 'Green-Gauss, least-squares gr...",NaN,NaN,NaN,NaN
